In [1]:
import pandas as pd
import numpy as np
import openrouteservice as ors
from dotenv import load_dotenv
from pyonemap import OneMap
import os
import requests
import json
import time
from collections import namedtuple
load_dotenv()
import folium

In [27]:
#get mrt data
mrt_df = pd.read_csv('../data/mrt_station_final.csv')
mrt_color = pd.read_csv('../data/mrt_station_colours.csv')
print(mrt_color.head())
coords = mrt_df[['Latitude', 'Longitude']].values.tolist()

# run this to start OTP: java -Xmx6G -jar otp-2.5.0-shaded.jar --load C:\Users\wjlwi\OneDrive\Desktop\Github\Public-Transportation-In-Singapore\Backend\otp  --serve

# Sample API request to the OTP isochrone endpoint
response = requests.get("http://localhost:8080/otp/traveltime/isochrone?batch=true&location=1.3521,103.895&time=2023-04-12T10:19:03%2B02:00&modes=WALK,TRANSIT&arriveBy=false&cutoff=30M17S")
# isochrone_geojson = response.json()


   S/N                 MRT.Name  Latitude   Longitude   Color
0    1    ESPLANADE MRT STATION  1.293658  103.855081  yellow
1    2   PAYA LEBAR MRT STATION  1.317199  103.892365  yellow
2    3  DHOBY GHAUT MRT STATION  1.298655  103.846194     red
3    4       DAKOTA MRT STATION  1.308548  103.889065  yellow
4    5     LAVENDER MRT STATION  1.307378  103.862768   green


Write a function for fetching isochrome. Configure params here

In [3]:

def fetch_isochrone(lat, lon, modes, cutoff="18M00S"):
    """Fetch isochrone GeoJSON for a given location."""
    url = "http://localhost:8080/otp/traveltime/isochrone"
    params = {
        "batch": "true",
        "location": f"{lat},{lon}",
        "time": "2023-04-12T10:19:03+02:00",
        "modes": modes,
        "arriveBy": "false",
        "cutoff": cutoff
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch isochrone for {lat},{lon}. Status code: {response.status_code}")
        return None


Visualise all isochrones

In [10]:
import plotly.graph_objs as go
import random

f = go.FigureWidget()
f.layout.hovermode = 'closest'
f.layout.hoverdistance = -1 #ensures no "gaps" for selecting sparse data
default_linewidth = 2
highlighted_linewidth_delta = 2

# just some traces with random data points  
num_of_traces = 5
random.seed = 42
for i in range(num_of_traces):
    y = [random.random() + i / 2 for _ in range(100)]
    trace = go.Scatter(y=y, mode='lines', line={ 'width': default_linewidth })
    f.add_trace(trace)

# our custom event handler
def update_trace(trace, points, selector):
    # this list stores the points which were clicked on
    # in all but one trace they are empty
    if len(points.point_inds) == 0:
        return
    for i,_ in enumerate(f.data):
        f.data[i]['line']['width'] = default_linewidth + highlighted_linewidth_delta * (i == points.trace_index)
    print(points.trace_index)

# we need to add the on_click event to each trace separately       
for i in range(len(f.data) ):
    f.data[i].on_click(update_trace)

# let's show the figure 
f

FigureWidget({
    'data': [{'line': {'width': 2},
              'mode': 'lines',
              'type': 'scatter',
              'uid': '747aba34-f3c1-4e26-8f57-9bf5c410263b',
              'y': [0.006876653617737372, 0.49889938341775597, 0.9737347378018458,
                    0.1339076628884629, 0.7210622307883127, 0.24700667212235394,
                    0.87499702071921, 0.7029415746008127, 0.9305613559642726,
                    0.41834271710000126, 0.5747393764766956, 0.4815755868966416,
                    0.156116276614373, 0.6048646686521458, 0.34642404503786906,
                    0.9668171760897527, 0.24952461854741215, 0.10599205605581097,
                    0.04659872708963875, 0.5480777983275658, 0.6495296153589536,
                    0.3698990037146608, 0.962481078399535, 0.3424002357037854,
                    0.7006764563375488, 0.2473610030149611, 0.7029334693955408,
                    0.6804721814194993, 0.4785983040999676, 0.9791876186047195,
                   

In [5]:
import plotly.graph_objects as go


fig = go.Figure(go.Scattermapbox(lat=[36,37,38],
                                 lon=[26,27,28],
                                 mode='markers',
                                 marker={'size':9,'color':'purple'},
                                 selected=go.scattermapbox.Selected(marker = {"color":"red", "size":25})
                                 )
                )

fig.update_layout(clickmode='event+select',
                  mapbox = {'style': "stamen-terrain",
                            'center': {'lon': 27.0, 'lat': 37.0 },
                            'zoom': 6},
                  margin = dict(l=0, r=0, t=0, b=0)
                  )

fig.show()

In [26]:
from plotly.subplots import make_subplots
f = go.FigureWidget(make_subplots(rows=2, cols=1))

trace1 = go.Scatter(x=[3, 4, 5], y=[10, 11, 12], name='trae1')
trace2 = go.Scatter(x=[6, 7, 8], y=[13, 14, 15])

f.add_trace(trace1, row=1, col=1)
f.add_trace(trace2, row=2, col=1)

def print_coordinates(trace, points, selector):
    if points.point_inds:  # Ensure a point is clicked
        # Extract the coordinates of the clicked marker
        print(trace)
        clicked_lon = trace.x[1]
        clicked_lat = trace.y[1]
        # Print the coordinates
        print(f"Clicked marker coordinates: Lat {clicked_lat}, Lon {clicked_lon}")

for trace in f.data:
    trace.on_click(print_coordinates)


a = 3
minutes = '%02d' % (3)
seconds = '%02d' % (4)
cutoff = minutes+"M"+seconds+"S"
print(cutoff)

03M04S


Plotting of the Graph of MRT Stations (Can click and toggle the size of the MRT Stations)

In [28]:
import plotly.graph_objects as go

# Create a Plotly figure
fig = go.Figure()
fig.layout.hovermode = 'closest'
default_linewidth = 2
highlighted_linewidth = 3

# Fetch and add isochrones for each MRT station
for index, (lat, lon) in enumerate(coords):
    mrt_name = mrt_df[(mrt_df['Latitude'] == lat) & (mrt_df['Longitude'] == lon)]['MRT.Name'].iloc[0]
    # Add isochrone as a trace
    color = mrt_color[(mrt_df['Latitude'] == lat) & (mrt_df['Longitude'] == lon)]['Color'].iloc[0]
    fig.add_trace(go.Scattermapbox(
        mode="markers",
        lon=[lon],
        lat=[lat],
        text = [mrt_name],
        marker=dict(size=10,color=color),
        selected=go.scattermapbox.Selected(marker = {"size":25}),
        unselected=go.scattermapbox.Unselected(marker = {"color":"grey", "size":5})
    ))
"""def click_handler(trace, points, selector):
    lat = trace['lat'][0]
    lon = trace['lon'][0]
    isochrone_geojson = fetch_isochrone(lat, lon, modes="TRANSIT")
    geometry = isochrone_geojson['geometry']
    polygon_color = mrt_color[(mrt_df['Latitude'] == lat) & (mrt_df['Longitude'] == lon)]['Color'].iloc[0]
    if 'coordinates' in geometry:
        coordinates = geometry['coordinates'][0][0]
        # Assuming coordinates are in [lon, lat] format
        lon_coords = [coord[0] for coord in coordinates]
        lat_coords = [coord[1] for coord in coordinates]
        fig.add_trace(go.Scattermapbox(
                    mode="lines",
                    lon=lon_coords,
                    lat=lat_coords,
                    name=f"Isochrone {lat}, {lon}",
                    line=dict(width=1, color=polygon_color),
                    fill = "toself",
                    marker=dict(size=0),
                    text=[mrt_name],  # Set text for hover
                    hoverinfo='text'
                ))
    
    print("Hi")
for trace in fig.data:
    trace.on_click(click_handler)"""
# Define layout for the map
fig.update_layout(
    clickmode='event+select',
    mapbox_style="carto-positron",
    mapbox_zoom=12,
    mapbox_center={"lat": coords[0][0], "lon": coords[0][1]},
    showlegend = False
)

fig

In [21]:

from dash import dcc
from dash import Dash, html, dcc
import json
app = Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(['Cycling', 'MRT', 'Bus', 'Cycling + Bus', 'Cycling + MRT'], 'Cycling'),
    dcc.Checklist(
        [
        {
            "label": html.Div(['Cycling'], style={'color': 'Gold', 'font-size': 20}),
            "value": "Cycling",
        },
        {
            "label": html.Div(['MRT'], style={'color': 'Red', 'font-size': 20}),
            "value": "MRT",
        },
        {
            "label": html.Div(['Bus'], style={'color': 'LightGreen', 'font-size': 20}),
            "value": "Bus",
        },
    ], value=['Cycling'],
    labelStyle={"display": "flex", "align-items": "center"},
    ),
    dcc.Slider(0, 20, 1, value=18),
    dcc.Graph(
        id='map',
        figure=fig 
    )
])

if __name__ == '__main__':
    app.run(debug=True)

In [9]:
from dash_extensions.javascript import assign
from dash import Dash
from dash_leaflet import Map, TileLayer

# Define JavaScript function to print coordinates.
js_function = """
function(e, ctx) {
    console.log(`You clicked at ${e.latlng.lat}, ${e.latlng.lng}.`);
}
"""

# Assign JavaScript function to the click event handler.
event_handlers = dict(click=assign(js_function))

# Create the Dash app.
app = Dash(__name__)
app.layout = Map(children=[TileLayer()], eventHandlers=event_handlers,
                 style={'height': '50vh'}, center=[56, 10], zoom=6)

if __name__ == '__main__':
    app.run_server()

In [29]:
import plotly.graph_objects as go

def fetch_isochrone(lat, lon, modes, cutoff="06M30S"):
    """Fetch isochrone GeoJSON for a given location."""
    url = "http://localhost:8080/otp/traveltime/isochrone"
    params = {
        "batch": "true",
        "location": f"{lat},{lon}",
        "time": "2023-06-01T18:00:00+02:00",
        "modes": modes,
        "arriveBy": "false",
        "cutoff": cutoff
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch isochrone for {lat},{lon}. Status code: {response.status_code}")
        return None
# Create a Plotly figure
fig = go.Figure()
color_sequence = ['blue', 'green', 'red', 'orange', 'purple', 'yellow'] 
# Fetch and add isochrones for each MRT station
for index, (lat, lon) in enumerate(coords):
    isochrone_geojson = fetch_isochrone(lat, lon, modes="TRANSIT")
    mrt_name = mrt_df[(mrt_df['Latitude'] == lat) & (mrt_df['Longitude'] == lon)]['MRT.Name'].iloc[0]
    if isochrone_geojson and 'features' in isochrone_geojson:
        # Access the first feature in the returned GeoJSON
        feature = isochrone_geojson['features'][0]
        if 'geometry' in feature:
            geometry = feature['geometry']
            if 'coordinates' in geometry:
                coordinates = geometry['coordinates'][0][0]
                # Assuming coordinates are in [lon, lat] format
                lon_coords = [coord[0] for coord in coordinates]
                lat_coords = [coord[1] for coord in coordinates]
                # Add isochrone as a trace
                color = color_sequence[index%len(color_sequence)]
                fig.add_trace(go.Scattermapbox(
                    mode="lines",
                    lon=lon_coords,
                    lat=lat_coords,
                    name=f"Isochrone {lat}, {lon}",
                    line=dict(width=1, color=color),
                    fill = "toself",
                    marker=dict(size=0),
                    text=[mrt_name],  # Set text for hover
                    hoverinfo='text'
                ))
                fig.add_trace(go.Scattermapbox(
                    mode="markers",
                    lon=[lon],
                    lat=[lat],
                    text=[mrt_name],
                    name=mrt_name,
                    marker=dict(size=10,color=color),
                    textposition="bottom center",
                ))


# Define layout for the map
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=12,
    mapbox_center={"lat": coords[0][0], "lon": coords[0][1]},
    showlegend = False
)

# Show the map
fig.show()

In [7]:

m = folium.Map(location=coords[0], zoom_start=12)

# Fetch and add isochrones for each MRT station
for lat, lon in coords:
    isochrone_geojson = fetch_isochrone(lat, lon,modes = "TRANSIT")
    if isochrone_geojson:
        folium.GeoJson(isochrone_geojson, name=f"Isochrone {lat}, {lon}").add_to(m)

# Add layer control and display the map
folium.LayerControl().add_to(m)
m


In [ ]:
#1 cycling range
#2 walking range
#3 bus + walking range
#https://github.com/gis-ops/routingpy

Saving isochrones for bus, walking, cycling.

In [19]:
# Assuming mrt_df is already defined
mrt_df['bus 30M isochrone'] = mrt_df.apply(lambda row: fetch_isochrone(row['Latitude'], row['Longitude'],modes = "BUS,WALK", cutoff="20M17S"), axis=1) #change modes to BUS/WALK
mrt_df['walking 30M isochrone'] = mrt_df.apply(lambda row: fetch_isochrone(row['Latitude'], row['Longitude'],modes = "WALK", cutoff="20M17S"), axis=1) #change modes to WALK
mrt_df['cycling 30M isochrone'] = mrt_df.apply(lambda row: fetch_isochrone(row['Latitude'], row['Longitude'],modes = "BICYCLE", cutoff="20M17S"), axis=1) #change modes to CYCLE


save data as .pkl to preserve json

In [20]:
#mrt_df.to_pickle("../data/mrt_df_with_isochrones.pkl")
mrt_df = pd.read_pickle("../data/mrt_df_with_isochrones.pkl")

In [27]:
import folium

m = folium.Map(location=[1.3521, 103.8198], zoom_start=12)

for index, row in mrt_df.iterrows():
    # Creating a popup message with a brief description or link
    popup_message = f"MRT Station: {index}<br>"
    popup_message += "Cycling 30M Isochrone available."  # Example message; customize as needed
    
    # Create and add a marker to the map
    folium.Marker(
        [row['Latitude'], row['Longitude']],
        popup=folium.Popup(popup_message, parse_html=True, max_width=450)
    ).add_to(m)
m




AttributeError: 'dict' object has no attribute 'str'